# Reddit Sentiment Scraping

**Author:** Charity Smith  
**Date:** 30 March 2025

### Notebook Description
This notebook collects and processes user-generated comments and posts from Reddit related to NBA three-point shooters and shooting trends over the past ten years. It includes scraping Reddit data, structuring collected comments, and performing preliminary sentiment analysis to gauge fan excitement or fatigue concerning the increase in three-point attempts in basketball.

In [13]:
import praw
import pandas as pd
from datetime import datetime
import os

In [4]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="pnIGUOh0Yawi9xh27vcqKw",
    client_secret="EWUxrEKZTy4iZUL-ROejUkvRa0wuKA",
    user_agent="NBA_ThreePoint_SentimentScraper/0.1 by Euler_groupie_13"
)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [5]:
# Define subreddits and keywords
subreddits = ['nba', 'nbadiscussion', 'warriors']
keywords = ['three point', 'three pointers', '3 pointer', '3-point', 'Steph Curry', 'shooting', 'too many threes']

# Empty list to store data
data = []

# Define the timeframe (approximate 10 years)
start_timestamp = int(datetime(2014, 1, 1).timestamp())

for sub in subreddits:
    subreddit = reddit.subreddit(sub)
    print(f"Scraping subreddit: {sub}")

    # Reddit API doesn't directly allow time-based searches; limit by count
    for post in subreddit.search(query=' OR '.join(keywords), limit=5000, sort='new'):
        if post.created_utc >= start_timestamp:
            post.comments.replace_more(limit=0)  # get all comments
            comments = post.comments.list()
            print(f"Collected post: {post.title}")

            # Save post data
            data.append({
                'type': 'post',
                'subreddit': sub,
                'title': post.title,
                'content': post.selftext,
                'created': datetime.fromtimestamp(post.created_utc),
                'score': post.score,
                'num_comments': post.num_comments
            })

            # Save comments data
            for comment in comments:
                data.append({
                    'type': 'comment',
                    'subreddit': sub,
                    'title': post.title,
                    'content': comment.body,
                    'created': datetime.fromtimestamp(comment.created_utc),
                    'score': comment.score,
                    'num_comments': None
                })


Scraping subreddit: nba
Collected post: [Marc Stein] “The Kroenkes have a fondness for Porter that stems from deep ties to their shared alma mater: Missouri. Sources with knowledge of DEN thinking have maintained for some time that they struggled to envision ownership ever approving a deal that would send Porter away from the franchise.”
Collected post: [OC] Tracking NBA MVP & Awards Votes (SGA 2 - Jokić 1) & Predicting the Awards Voters
Collected post: [NBA.com] Kia MVP Ladder: In final analysis, it's Shai Gilgeous-Alexander over Nikola Jokić
Collected post: Payton Pritchard vs Ty Jerome season comparison - Why is Pritchard the runaway favorite for sixth man of the year?
Collected post: Where do these power forwards rank amongst each other: Bam Adebayo, Jaren Jackson Jr, and Pascal Siakam
Collected post: [Amick] It was Monday afternoon in Denver, and the Nuggets’ GM Booth had reached his breaking point with Malone, whose team was on a four-game slide. The Nuggets’ locker room was a me

In [6]:
# Convert data to DataFrame
df_reddit = pd.DataFrame(data)

# Quickly inspect your DataFrame
df_reddit.head()

,type,subreddit,title,content,created,score,num_comments
0,post,nba,[Marc Stein] “The Kroenkes have a fondness for...,Source: https://marcstein.substack.com/p/the-i...,2025-04-12 10:15:12,596,179.0
1,comment,nba,[Marc Stein] “The Kroenkes have a fondness for...,the kroenkes won't trade MIchael Porter Jr. be...,2025-04-12 10:18:52,1328,NaN
2,comment,nba,[Marc Stein] “The Kroenkes have a fondness for...,lol reminds me of the rumor that Jeanie would ...,2025-04-12 10:17:32,376,NaN
3,comment,nba,[Marc Stein] “The Kroenkes have a fondness for...,This is such a stupid reason lol,2025-04-12 10:18:46,146,NaN
4,comment,nba,[Marc Stein] “The Kroenkes have a fondness for...,"Are you kidding me, fucking Michael Porter Jun...",2025-04-12 13:47:59,20,NaN


In [12]:
os.makedirs("../data/reddit_data", exist_ok=True)
df_reddit.to_csv('../data/reddit_data/nba_three_point_sentiment.csv', index=False)